In [15]:
import pandas as pd
from gensim.models import doc2vec

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords

import string

In [3]:
dftrain = pd.read_csv('train_tokens_facts_number.csv', encoding = 'utf-8')
dftest = pd.read_csv('test_tokens_facts_number.csv', encoding = 'utf-8')

In [16]:
wd = {}

def word_tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [i.lower() for i in tokens if ( i not in string.punctuation+'—' )]
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
    tokens = [i for i in tokens if ( i not in stop_words )]
    stems = []
    for item in tokens:
        if item in wd.keys():
            stems.append(wd[item])
        else:
            w = SnowballStemmer(language='russian').stem(item)
            wd[item] = w
            stems.append(w)
    return stems

In [17]:
questions = dftrain.tokens_question.tolist()
paragraphs = dftrain.tokens_paragraph.tolist() + dftest.tokens_paragraph.tolist()
texts = paragraphs + questions

labels = []
for label in dftrain['paragraph_id'].tolist():
    labels.append('train_par_id_%s' % label)
for label in dftest['paragraph_id'].tolist():
    labels.append('test_par_id_%s' % label)
for label in dftrain['question_id'].tolist():
    labels.append('train_quest_id_%s' % label)

class LabeledLineSentence(object):

    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.LabeledSentence(words=word_tokenize(doc),
                                         tags=[self.labels_list[idx]])

sentences = LabeledLineSentence(texts, labels)
it = sentences.__iter__()

model = gensim.models.Doc2Vec(size=300, window=10, min_count=10, workers=12, alpha=0.03, min_alpha=0.025)
model.build_vocab(it)

for epoch in range(5):
    print('training epoch %s'%epoch)
    sentences = LabeledLineSentence(texts, labels)
    it = sentences.__iter__()
    model.train(it, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha


training epoch 0
training epoch 1
training epoch 2
training epoch 3
training epoch 4


'Какой ГОСТ предусматривает термин пиксель как единственно возможный для использования в области применения указанного стандарта?'

In [19]:
fname = 'saved_model.dat'
model.save(fname)

In [ ]:
model = gensim.models.Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [21]:
model.docvecs.most_similar('train_par_id_1436')

[('train_quest_id_26142', 0.4801800847053528),
 ('train_par_id_7141', 0.43672940135002136),
 ('train_quest_id_44246', 0.4327927529811859),
 ('train_quest_id_91308', 0.42102304100990295),
 ('train_quest_id_26759', 0.42047473788261414),
 ('train_quest_id_31677', 0.4202239215373993),
 ('train_quest_id_64345', 0.4155823886394501),
 ('train_quest_id_96648', 0.4148680567741394),
 ('train_quest_id_59875', 0.4145047664642334),
 ('train_quest_id_15207', 0.41430264711380005)]

In [26]:
dftrain[dftrain.paragraph_id==1436].paragraph.values[0]
#['paragraph'].values[0])

'Относительно нормативности использовании термина в форме пиксел либо пиксель имеются различные мнения. Так Русский орфографический словарь РАН [4] квалифицирует форму пиксел как общеупотребительную, а форму пиксель как характерную разговорной профессиональной или разговорной и профессиональной речи (в сокращениях словаря нет расшифровки для разг. проф. речи, но есть отдельно разг. — разговорное, проф. — профессиональное[5]; однозначной расшифровки этого определения не даёт и справочная служба русского языка на портале Грамота.ру[6]). С другой стороны, действующий ГОСТ 27459-87[7] предусматривает термин пиксель как единственно возможный для использования в области применения указанного стандарта (компьютерная графика) и который является обязательным для применения в документации и литературе всех видов, входящих в сферу действия стандартизации или использующих результаты этой деятельности . При этом ГОСТ 27459-87 под термином пиксель понимает наименьший элемент поверхности визуализации

In [25]:
dftrain[dftrain.question_id==26142].question.values[0]

'Какой термин пиксел или пиксель действующий ГОСТ 27459-87 предусматривает как единственно возможный для использования в области применения указанного стандарта?'

In [35]:
dftrain[dftrain.question_id==26142].question.values[0]

'Какой термин пиксел или пиксель действующий ГОСТ 27459-87 предусматривает как единственно возможный для использования в области применения указанного стандарта?'